<a href="https://colab.research.google.com/github/runfish5/tiny-tutorials/blob/main/discord-transcription-bot/jupyter-notebooks/Discord_bot_notebook_Nvidia_CPU_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎙️✍️ Voice Transcription Bot

#<font color='00BBF9'> Step 1: 🚦 Run this for overall setup

In [1]:
#@title Install requirements
%%capture
'''
This is more cumbersome than necessary, but
this way it creates a requirements.txt
which is desirable.
'''

content = """py-cord
python-dotenv"""
# openai-whisper""" # openai-whisper is slow to install...
# openai-whisper is not required if working with whisperx


with open("requirements.txt", "w") as file:
    file.write(content)

print("requirements.txt has been created.")

!pip install -r ./requirements.txt
!pip install PyNaCl
!pip install nest_asyncio


%cd /content/
!rm -rf /content/tiny-tutorials
!git clone https://github.com/runfish5/tiny-tutorials.git


In [2]:
#@title script definitions

# cur_asr_model = 'WHISPERX' # Go to: https://github.com/runfish5/tiny-tutorials/tree/main/discord-transcription-bot/jupyter-notebooks
cur_asr_model = 'NVIDIA'     # Go to: https://github.com/runfish5/tiny-tutorials/tree/main/discord-transcription-bot/jupyter-notebooks



'''
for jupyter notebook text:

  00BBF9    -     Deep sky blue
  00A115    -     Pigment green
  00A6A6    -     Light sea green
'''

GREEN = "\033[92m"
YELLOW = "\033[93m"
BLUE = "\033[94m"
CYAN = "\033[96m"
RESET = "\033[0m"
MAGENTA = '\033[35m'
RED = '\033[31m'
ORANGE = "\033[38;5;208m"


RESET = '\033[0m'

# <font color='00BBF9'> Step 2: 🔌 Turn on bot
- Set your `DISCORD_BOT_TOKEN` in the first cell

In [3]:
#@title import packages; Load environment variables
import tempfile,  os, discord, asyncio, nest_asyncio
nest_asyncio.apply()

from pprint import pprint
from dotenv import load_dotenv
# import os
# print("Environment variables:", os.environ)

%cd /content/tiny-tutorials/discord-transcription-bot

env_content = """DISCORD_BOT_TOKEN=...
"""

with open(".env", "w") as env_file:
    env_file.write(env_content)

!pwd
load_dotenv()
print("'.env' file loaded successfully!")

/content/tiny-tutorials/discord-transcription-bot
/content/tiny-tutorials/discord-transcription-bot
'.env' file loaded successfully!


In [4]:
token = os.getenv("DISCORD_BOT_TOKEN")

print(f"'{token}'")

'...'


In [ ]:
#@title Get bot status

def get_bot_status():
  if 'bot_task' not in globals():

      print("ERROR: bot_task not in globals()")

  elif bot_task.done():
      print("Bot is NOT (!) running. 🛑")
  else:
      print("Bot is RUNNING. 🔴 ")


# get_bot_status()
get_bot_status()


Bot is NOT (!) running. 🛑


In [ ]:
#@title <font color='magenta'> 📡 Run main.py and keep-alive task
# (the fancy way)

%cd /content
exec(open("/content/tiny-tutorials/discord-transcription-bot/main.py").read())

# Global stop flag
running = True

async def keep_alive():
    global running
    while running:
        await asyncio.sleep(1)  # Keep the loop alive without blocking

# Start the bot and the keep-alive task
loop            = asyncio.get_event_loop()
bot_task        = loop.create_task(run_bot())         # `run_bot()` is defined in the `main.py`
keep_alive_task = loop.create_task(keep_alive())

get_bot_status()

/content
This script is meant to be run from the notebook.
Bot is RUNNING. 🔴 


In [ ]:
#@title <font color='magenta'>bot: off-switch

await bot.close()

get_bot_status()
'''
Set the stop flag
'''
running = False

Bot is NOT (!) running. 🛑


In [ ]:
#@title 💾 zip it

'''
We zip it for downloading.
(Google Colab doesn't support download and upload of directories.)
'''
from datetime import datetime

current_date = datetime.now().strftime("%y%m%d")

base_zip_filename = f"{current_date}_recordings.zip"

# Check if file exists and add version number if needed
i = 1
zip_filename = base_zip_filename
while os.path.exists(zip_filename):
    zip_filename = f"{current_date}_recordings_{i}.zip"
    i += 1

# Zip the recordings directory with the dated filename
!zip -r {zip_filename} recordings

print(f"{MAGENTA}Created zip file: {zip_filename}{RESET}")

  adding: recordings/ (stored 0%)
Created zip file: 250514_recordings.zip


#<font color='00BBF9'>Step 3: ⚡ ASR /  Automatic Speech Recognition 🎙️✍️

In [ ]:
%cd /content

/content


https://discordpy.readthedocs.io/en/latest/discord.html

Direct for discord using javascript: https://github.com/Rei-x/discord-speech-recognition?tab=readme-ov-file

##<font color='00A115'>Nvidia ASR 2025
1. install (~3 min 30 sec - model: parakeet-tdt-0.6b-v2)
2. restart session

In [ ]:
!nvidia-smi -q


==============NVSMI LOG==============

Timestamp                                 : Wed May 14 17:20:20 2025
Driver Version                            : 550.54.15
CUDA Version                              : 12.4

Attached GPUs                             : 1
GPU 00000000:00:03.0
    Product Name                          : NVIDIA L4
    Product Brand                         : NVIDIA
    Product Architecture                  : Ada Lovelace
    Display Mode                          : Enabled
    Display Active                        : Disabled
    Persistence Mode                      : Disabled
    Addressing Mode                       : None
    MIG Mode
        Current                           : N/A
        Pending                           : N/A
    Accounting Mode                       : Disabled
    Accounting Mode Buffer Size           : 4000
    Driver Model
        Current                           : N/A
        Pending                           : N/A
    Serial Number          

In [ ]:
#@title 💤 Installing nemo requirements. ( ~3 minutes...)

!pip install "numpy<2.0.0" # (Won't be necessary for next parakeet release)
!pip install -U nemo_toolkit['asr']

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 112.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.4/79.4 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of onnx to determin

<font color='red'> RESTART SESSION

Only variable to set:
- `cur_asr_model = 'NVIDIA'`
- colors

In [ ]:
#@title import ASR model (nvidia) and define as `asr_model`
if cur_asr_model == "NVIDIA":
  import nemo.collections.asr as nemo_asr

  ''' Huggingface Access required ("HF_TOKEN". Grant access)'''
  # In google colab in the leftside menu bar is a icon of a key.
  # Click on it and select Add new secret.
  # Name it HF_TOKENand paste your new Huggingface token in there.
  # Toggle theNotebook access` so it shows a tick.

  asr_model = nemo_asr.models.ASRModel.from_pretrained(model_name="nvidia/parakeet-tdt-0.6b-v2")

  #get Sample from Nvidia:
  # !wget https://dldata-public.s3.us-east-2.amazonaws.com/2086-149220-0033.wav

parakeet-tdt-0.6b-v2.nemo:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

[NeMo I 2025-05-14 17:57:49 nemo_logging:393] Tokenizer SentencePieceTokenizer initialized with 1024 tokens


[NeMo W 2025-05-14 17:57:49 nemo_logging:405] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    use_lhotse: true
    skip_missing_manifest_entries: true
    input_cfg: null
    tarred_audio_filepaths: null
    manifest_filepath: null
    sample_rate: 16000
    shuffle: true
    num_workers: 2
    pin_memory: true
    max_duration: 40.0
    min_duration: 0.1
    text_field: answer
    batch_duration: null
    use_bucketing: true
    bucket_duration_bins: null
    bucket_batch_size: null
    num_buckets: 30
    bucket_buffer_size: 20000
    shuffle_buffer_size: 10000
    
[NeMo W 2025-05-14 17:57:49 nemo_logging:405] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config :

[NeMo I 2025-05-14 17:57:49 nemo_logging:393] PADDING: 0
[NeMo I 2025-05-14 17:57:54 nemo_logging:393] Using RNNT Loss : tdt
    Loss tdt_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0, 'durations': [0, 1, 2, 3, 4], 'sigma': 0.02, 'omega': 0.1}
[NeMo I 2025-05-14 17:57:54 nemo_logging:393] Using RNNT Loss : tdt
    Loss tdt_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0, 'durations': [0, 1, 2, 3, 4], 'sigma': 0.02, 'omega': 0.1}
[NeMo I 2025-05-14 17:57:54 nemo_logging:393] Using RNNT Loss : tdt
    Loss tdt_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0, 'durations': [0, 1, 2, 3, 4], 'sigma': 0.02, 'omega': 0.1}
[NeMo I 2025-05-14 17:57:57 nemo_logging:393] Model EncDecRNNTBPEModel was successfully restored from /root/.cache/huggingface/hub/models--nvidia--parakeet-tdt-0.6b-v2/snapshots/50aec6a056e85b9f95b612df08a2bddc55b58714/parakeet-tdt-0.6b-v2.nemo.


In [ ]:
#@title Optional: Testing the model using the Nvidia test file
%cd /content
# # !wget https://dldata-public.s3.us-east-2.amazonaws.com/2086-149220-0033.wav

# if cur_asr_model == 'NVIDIA':
#   !pwd
#   output = asr_model.transcribe(['2086-149220-0033.wav']) # 4 sec.
# #   print(output[0].text)

# =======
# if cur_asr_model == "NVIDIA":

#   output = asr_model.transcribe(['/content/2086-149220-0033.wav'], timestamps=True)
#   # by default, timestamps are enabled for char, word and segment level
#   word_timestamps = output[0].timestamp['word'] # word level timestamps for first sample
#   segment_timestamps = output[0].timestamp['segment'] # segment level timestamps
#   char_timestamps = output[0].timestamp['char'] # char level timestamps

#   for stamp in segment_timestamps:
#       print(f"{stamp['start']}s - {stamp['end']}s : {stamp['segment']}")

/content


In [ ]:
#@title <font color='magenta'>Optional: You can download a test `/recordings/` directory with this cell.
'''
not required if you recorded your discord session
'''
# import gdown
# import zipfile
# import os
# import shutil

# # Replace with your shared link
# shared_link = "https://drive.google.com/file/d/1tEu6xqF9DMQ9FEcXZ4guG3uymxiVEtcZ/view?usp=sharing"
# shared_link = "https://drive.google.com/file/d/1ouoroEO0tbBGXs4LNDCF9vwaM9306jZF/view?usp=sharing"
# # Extract the file ID from the shared link
# file_id = shared_link.split("/")[5]

# # Download the file
# output_path = "/content/downloaded_file.zip"

# gdown.download(f"https://drive.google.com/uc?id={file_id}", output_path, quiet=False)

# # Unzip the file
# with zipfile.ZipFile(output_path, 'r') as zip_ref:
#     zip_ref.extractall("/content/extracted_files")

# # ==================================================
# #@title COPY THE RECORDINGS (.wav)

# recordings_dir = "recordings"

# # Check if 'recordings' directory exists and is empty (or doesn't exist at all)
# if not os.path.exists(recordings_dir) or not os.listdir(recordings_dir):
#     # Create the directory if it doesn't exist
#     os.makedirs(recordings_dir, exist_ok=True)

#     # Copy the 'recordings' directory from extracted files
#     shutil.copytree("/content/extracted_files/recordings", recordings_dir, dirs_exist_ok=True)


'\nnot required if you recorded your discord session\n'

##<font color='00A115'>Step 3: Transcribe Audio Files 🔄

In [ ]:
#@title import model_config
%cd /content/tiny-tutorials/discord-transcription-bot
import model_config

if cur_asr_model == "NVIDIA":
  print("USING NVIDIA")
  # model_nvidia = nemo_asr.models.EncDecCTCModelBPE.from_pretrained("nvidia/stt_en_conformer_ctc_large")
  # asr_model = model_nvidia
  model_config.model = asr_model

else:
  # import nest_asyncio
  # nest_asyncio.apply()

  # import asyncio
  # import whisper

  # Load the model
  # model = whisper.load_model("base")
  # print("Whisper model loaded successfully!")

  # Assign the model to the configuration
  # model_config.model = whisper.load_model("base")
  model_config.model = model


  # device = "cpu"
  # model = whisperx.load_model("large-v2", device, compute_type="float32")


/content/tiny-tutorials/discord-transcription-bot
USING NVIDIA


In [ ]:
#@title define convert nvidia_to_whisperx():
def nvidia_to_whisperx(input_data):

    result = {"segments": []}

    # First, process segments
    for segment in input_data.get("segments", []):
        new_segment = {
            "start": segment["start"],
            "end": segment["end"],
            "text": segment["segment"],
            "words": []
        }

        # Find all words that belong to this segment
        segment_start_offset = segment["start_offset"]
        segment_end_offset = segment["end_offset"]

        for word in input_data.get("words", []):
            # Check if word is within this segment's offset range
            if segment_start_offset <= word["start_offset"] and word["end_offset"] <= segment_end_offset:
                new_word = {
                    "word": word["word"],
                    "start": word["start"],
                    "end": word["end"],
                    # "score": 0.8  # Adding default score since it's in target but not in source
                }
                new_segment["words"].append(new_word)

        result["segments"].append(new_segment)

    return result

# # Example usage
# import json

# initial_data = {
#     "segments": [
#         {
#             "segment": "Hello, this is the second account.",
#             "start_offset": 2,
#             "end_offset": 32,
#             "start": 0.16,
#             "end": 2.56
#         },
#         # ... other segments ...
#     ],
#     "words": [
#         {
#             "word": "Hello,",
#             "start_offset": 2,
#             "end_offset": 8,
#             "start": 0.16,
#             "end": 0.64
#         },
#         # ... other words ...
#     ]
# }

# converted = nvidia_to_whisperx(initial_data)
# print(json.dumps(converted, indent=2))

In [ ]:
#@title <font color='00A115'>✍ transcribe_all_recordings

import os
import json
import time
import model_config
import subprocess
import soundfile as sf

CYAN = "\033[96m"
RED = '\033[31m'
RESET = "\033[0m"


def transcribe_with_nvidia_parakeet(audio_path):

      # Construct the cleaned file path
      cleaned_path = audio_path.replace(".wav", "_cleaned.wav")

      # Run ffmpeg to re-encode the file to single-channel, 16kHz
      subprocess.run([
          "ffmpeg", "-y",          # Overwrite without prompting
          "-i", audio_path,        # Input file
          "-ac", "1",              # Force mono audio
          "-ar", "16000",          # Force 16kHz sample rate
          cleaned_path
      ], check=True)

      # Verify that the cleaned file has audio data using soundfile
      data, sr = sf.read(cleaned_path)
      if len(data) == 0:
          raise ValueError(f"Converted audio file {cleaned_path} has no data!")
      else:
          print(f"Converted file has {len(data)} samples at {sr} Hz.")

      # Use the cleaned file for transcription
      audio_path = cleaned_path
      print(f"{CYAN}{audio_path}{RESET}")
      output = asr_model.transcribe([audio_path], timestamps=True)

      for stamp in output[0].timestamp['segment']:
          print(f"{CYAN}{stamp['start']}s - {stamp['end']}s : {stamp['segment']}{RESET}")

      # transcription = asr_model.transcribe(['/content/2086-149220-0033.wav'], timestamps=True)
      # word_timestamps = output[0].timestamp['word'] # word level timestamps for first sample
      # segment_timestamps = output[0].timestamp['segment'] # segment level timestamps
      # char_timestamps = output[0].timestamp['char'] # char level timestamps

      transcription_text = nvidia_to_whisperx(
          {
          "segments": output[0].timestamp['segment'],
          "words" :output[0].timestamp['word']
          }
      )
      return transcription_text


def transcribe_with_nvidia_whisperx(audio_path):
    audio = whisperx.load_audio(audio_path)
    transcription = model_config.model.transcribe(audio)
    transcription = whisperx.align(transcription["segments"],
                                    model_a,
                                    metadata_model_a,
                                    audio,
                                    device,
                                    return_char_alignments=False)
    transcription_text = transcription if transcription.get('segments', None) else transcription["text"]

    return transcription_text




def transcribe_all_recordings(skip_existing=True):
    """
    Transcribes all audio recordings in the 'recordings' directory using the Whisper model.
    Saves transcriptions as text files and updates metadata.json for each user.
    If skip_existing is True, it will not re-transcribe already existing transcription files.
    """
    model_config.model = model_config.model  # Ensure model is set (redundant here if already set)

    recordings_dir = "recordings"
    if not os.path.exists(recordings_dir):
        print("Error: The 'recordings' directory does not exist.")
        return

    for user_id in os.listdir(recordings_dir):
        user_dir = os.path.join(recordings_dir, user_id)
        if not os.path.isdir(user_dir):
            continue

        metadata_path = os.path.join(user_dir, "metadata.json")
        if not os.path.exists(metadata_path):
            print(f"Warning: No metadata.json found for user {user_id}. Skipping.")
            continue

        with open(metadata_path, "r") as f:
            metadata = json.load(f)

        for recording in metadata.get("recordings", []):
            audio_file = recording.get("audio_file")
            if not audio_file:
                print(f"Warning: No audio_file specified in metadata for user {user_id}. Skipping entry.")
                continue

            audio_path = os.path.join(user_dir, audio_file)
            if not os.path.exists(audio_path):
                print(f"Warning: Audio file {audio_path} not found. Skipping.")
                continue

            transcription_file = f"{os.path.splitext(audio_file)[0]}_transcription.json"
            transcription_path = os.path.join(user_dir, transcription_file)

            if skip_existing and os.path.exists(transcription_path):
                print(f"Skipping existing transcription for {audio_file}.")
                continue

            try:
                start_time = time.time()

                if cur_asr_model == "WHISPERX":
                    transcription_text = transcribe_with_nvidia_parakeet(
                       audio_path=audio_path
                    )
                elif cur_asr_model == "NVIDIA":
                    transcription_text = transcribe_with_nvidia_parakeet(
                        audio_path=audio_path
                    )
                else:
                  raise ValueError(f"Unknown ASR model: {cur_asr_model}")

                end_time = time.time()
                duration = end_time - start_time

            except Exception as e:
                print(f"Error: Failed to transcribe {audio_path}: {str(e)}")
                continue

            with open(transcription_path, "w") as f:
                f.write(json.dumps(transcription_text, indent=4, ensure_ascii=False))

            recording["transcription_file"] = transcription_file
            recording["transcription_time_seconds"] = round(duration, 2)
            print(f"{RED}Transcribed {audio_file} in {duration:.2f} seconds.{RESET}")

        with open(metadata_path, "w") as f:
            json.dump(metadata, f, indent=4)

        print(f"Success: Transcriptions completed for user {user_id}")

# Example usage:
%cd /content
# transcribe_all_recordings(skip_existing=True)  # Skips existing transcriptions
transcribe_all_recordings(skip_existing=False)  # Re-transcribes all files


/content
Converted file has 10097276 samples at 16000 Hz.
recordings/runfishrun/session_20250514_17M51_cleaned.wav
[NeMo I 2025-05-14 17:57:58 nemo_logging:393] Timestamps requested, setting decoding timestamps to True. Capture them in Hypothesis object,                         with output[0][idx].timestep['word'/'segment'/'char']
[NeMo I 2025-05-14 17:57:58 nemo_logging:393] Using RNNT Loss : tdt
    Loss tdt_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0, 'durations': [0, 1, 2, 3, 4], 'sigma': 0.02, 'omega': 0.1}


[NeMo W 2025-05-14 17:57:58 nemo_logging:405] `include_duration` is not implemented for CUDA graphs
Transcribing: 100%|██████████| 1/1 [00:09<00:00,  9.30s/it]


4.72s - 7.36s : Yeah, I don't know what's your stream on here.
20.080000000000002s - 22.240000000000002s : Yeah, so how is that going?
70.88s - 73.92s : Yeah, I just get this bot done.
74.88s - 85.68s : I also think about how to add like text to the video, like to make us to make it a mask or something like that to put it over there.
89.04s - 91.92s : This is gonna be difficult, I don't know.
114.56s - 138.32s : Okay, but you can do like how long can the videos be on Canvas so Canvas is like really not just for ports, but also for long videos or what?
150.72s - 156.8s : Okay, yeah, this is it's so much work.
173.36s - 173.68s : Yeah.
189.28s - 189.6s : Yeah.
206.72s - 207.84s : Up to one terabyte.
207.92000000000002s - 209.52s : So that's like uh four.
210.08s - 210.64000000000001s : Yeah, okay.
212.08s - 230.8s : I mean, that's yeah, uh, yeah, but not too long.
231.12s - 241.52s : I mean, if you have a normal HD movie like 1080, uh, two hours is about is at least four gigabytes.
264.1

[NeMo W 2025-05-14 17:58:08 nemo_logging:405] `include_duration` is not implemented for CUDA graphs
Transcribing: 100%|██████████| 1/1 [00:08<00:00,  8.43s/it]

0.08s - 1.04s : No, it's okay.
1.68s - 9.68s : Just working on the video, you know, trying to dissolve and put some sliders in a better position.
15.68s - 20.400000000000002s : I'm currently helping myself with ChatGPT due to it's new for me.
21.44s - 25.12s : But I'm seriously thinking to change a different program after this.
25.36s - 29.36s : Like that's the one and only project I'm doing with Canvas.
30.88s - 47.04s : It's like because when you have slides and then you have you make different slides in between those slides, there's a millisecond of a screen flicker which quickly flashes and like it's really making a weird behavior.
47.36s - 54.32s : So it's asking, it's telling me to dissolve in between with a little plus, but there is no plus.
54.480000000000004s - 58.96s : Like GPT tells me clearly there's a little plus, but that's what it is.
59.120000000000005s - 59.6s : Like, yeah.
60.160000000000004s - 61.76s : What are you working on?
70.96000000000001s - 71.52s : Yes.
78.88

- CPU:
    - 204 sec recording: 147 sec to transcribe
    - 60 sec          : 32 sec to transcribe

Adding speaker identification to nvidiaparakeet
- https://huggingface.co/nvidia/parakeet-tdt-0.6b-v2/discussions/16

In [ ]:
#@title define refine segment splits

def refine_split_segments(transcript_data, time_threshold=5.0):
    """
    Splits transcript segments when there's a gap larger than the threshold between consecutive words.

    Args:
        transcript_data (list or dict): Either a list of transcript segments or a single segment dict
        time_threshold (float): Maximum allowed time gap between words (in seconds)

    Returns:
        list: A list of segment dictionaries in the same format as the input
    """
    # Handle input that's already a list or convert single dict to list
    segments_list = transcript_data if isinstance(transcript_data, list) else [transcript_data]
    result = []

    # Process each segment
    for segment in segments_list:
        # Handle edge cases
        if not segment or 'words' not in segment or len(segment.get('words', [])) <= 1:
            result.append(segment)
            continue

        words = segment['words']

        # Find breakpoints where time gap exceeds threshold
        breakpoints = [i for i in range(1, len(words))
                      if words[i]['start'] - words[i-1]['end'] > time_threshold]

        # If no breakpoints found, keep original segment
        if not breakpoints:
            result.append(segment)
            continue

        # Create segments based on breakpoints
        breakpoints = [0] + breakpoints + [len(words)]  # Add start and end indices

        # Create all segments in one loop
        for i in range(len(breakpoints) - 1):
            start_idx = breakpoints[i]
            end_idx = breakpoints[i+1]
            segment_words = words[start_idx:end_idx]

            result.append({
                "start": segment_words[0]['start'],
                "end": segment_words[-1]['end'],
                "text": " ".join(w['word'] for w in segment_words),
                "words": segment_words
            })

    return result


# Example usage:

# example_data = [{
#     "start": 103.68,
#     "end": 166.88,
#     "text": "Let's see, I heard the echo that was pretty hard.",
#     "words": [
#         {"word": "Let's", "start": 103.68, "end": 104.32000000000001},
#         {"word": "see", "start": 104.32000000000001, "end": 104.56},
#         {"word": "Yeah,", "start": 153.12, "end": 153.44},
#         {"word": "I", "start": 153.76, "end": 154.0},
#         {"word": "heard", "start": 154.0, "end": 154.48},
#         {"word": "the", "start": 154.48, "end": 154.72},
#         {"word": "echo", "start": 154.72, "end": 155.28},
#         {"word": "that", "start": 155.52, "end": 155.76},
#         {"word": "was", "start": 155.76, "end": 156.0},
#         {"word": "pretty", "start": 156.0, "end": 156.48},
#         {"word": "hard.", "start": 166.48, "end": 166.88}
#     ]
# }]

# # Split with a threshold of 5 seconds
# results = refine_split_segments(example_data, 5.0)

# # Print the results
# print(f"Found {len(results)} segments:")
# for i, segment in enumerate(results):
#     print(f"\nSegment {i+1}:")
#     print(f"  Start: {segment['start']}")
#     print(f"  End: {segment['end']}")
#     print(f"  Text: {segment['text']}")
#     print(f"  Words: {len(segment['words'])}")

In [ ]:
#@title calculate_time_offset & adjust_transcription_times
import json
import os
from datetime import datetime

# Define color codes for output (optional, assuming they’re defined elsewhere if needed)
GREEN = '\033[92m'
RESET = '\033[0m'

def load_json_file(file_path):
    """Loads a JSON file and returns its contents."""
    try:
        with open(file_path, 'r') as f:
            return json.load(f)
    except (FileNotFoundError, json.JSONDecodeError) as e:
        print(f"Error loading {file_path}: {e}")
        return None

def calculate_time_offset(recording, reference_time):
    """
    Calculates time difference between user start time and the reference time in seconds.

    Args:
        recording (dict): Recording metadata with user_recording_start_time.
        reference_time (datetime): The unified reference time (earliest session_start_time).

    Returns:
        float: Time offset in seconds, or None if parsing fails.
    """
    try:
        user_dt = datetime.strptime(recording["user_recording_start_time"], "%Y-%m-%d %H:%M:%S")
        offset = (user_dt - reference_time).total_seconds()
        print(f"{GREEN}total_seconds offset for {recording['audio_file']}: {offset}{RESET}")
        return offset
    except ValueError as e:
        print(f"Time parsing failed for {recording['audio_file']}: {e}")
        return None

def get_transcription_path(user_dir, audio_file):
    """Constructs and validates the transcription file path."""
    audio_base = os.path.splitext(audio_file)[0]
    trans_file = f"{audio_base}_transcription.json"
    trans_path = os.path.join(user_dir, trans_file)
    if not os.path.exists(trans_path):
        print(f"Missing transcription: {trans_path}")
        return None
    return trans_path

def adjust_timestamps(transcription, time_offset):
    """Adjusts all start and end times in the transcription by the offset."""
    for segment in transcription["segments"]:
        segment["start"] += time_offset
        segment["end"] += time_offset
        for word in segment["words"]:
            word["start"] = word.get("start", 0) + time_offset
            word["end"] = word.get("end", 0) + time_offset

def save_adjusted_transcription(trans_path, transcription):
    """Saves the adjusted transcription to a new file."""
    new_trans_path = trans_path.replace(".json", "_adjusted.json")
    with open(new_trans_path, 'w') as f:
        json.dump(transcription, f, indent=4)
    print(f"Saved adjusted transcription to {new_trans_path}")

def adjust_transcription_times(metadata_path, reference_time):
    """
    Shifts timing in transcription files to align with the earliest session start time.

    Args:
        metadata_path (str): Path to metadata JSON file (e.g., 'recordings/<user_id>/metadata.json').
        reference_time (datetime): The earliest session_start_time across all sessions.
    """
    metadata = load_json_file(metadata_path)
    if not metadata:
        return

    user_dir = os.path.dirname(metadata_path)
    for recording in metadata["recordings"]:
        time_offset = calculate_time_offset(recording, reference_time)
        if time_offset is None:
            continue

        trans_path = get_transcription_path(user_dir, recording["audio_file"])
        if not trans_path:
            continue

        transcription = load_json_file(trans_path)

        if not transcription:
            continue

        transcription["segments"] = refine_split_segments(
            transcription["segments"],
            time_threshold=5.0
        )

        adjust_timestamps(transcription, time_offset)
        save_adjusted_transcription(trans_path, transcription)

# Process all user directories
base_dir = "/content/tiny-tutorials/discord-transcription-bot/recordings"
base_dir = "/content/recordings"

user_dirs = [d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))]

# Step 1: Find the earliest session_start_time (T0)
session_start_times = []
for user_dir in user_dirs:
    metadata_path = os.path.join(base_dir, user_dir, "metadata.json")
    metadata = load_json_file(metadata_path)
    if metadata:
        for recording in metadata["recordings"]:
            session_start_times.append(recording["session_start_time"])

if not session_start_times:
    print("No session start times found. Cannot proceed with adjustments.")
else:
    # Convert to datetime objects and find the minimum
    session_start_datetimes = [datetime.strptime(st, "%Y-%m-%d %H:%M:%S") for st in session_start_times]
    T0 = min(session_start_datetimes)
    start_time_str = str(T0)
    print(f"Earliest session start time (T0): {T0}")

    # Step 2: Adjust transcriptions using T0 as the unified reference
    for user_dir in user_dirs:
        metadata_path = os.path.join(base_dir, user_dir, "metadata.json")
        adjust_transcription_times(metadata_path, T0)

Earliest session start time (T0): 2025-05-14 17:46:17
total_seconds offset for session_20250514_17M51.wav: 2.0
Saved adjusted transcription to /content/recordings/runfishrun/session_20250514_17M51_transcription_adjusted.json
total_seconds offset for session_20250514_17M51.wav: 10.0
Saved adjusted transcription to /content/recordings/Unknown/session_20250514_17M51_transcription_adjusted.json


In [ ]:
# @title create_unified_transcription_object
import os
import json
from datetime import datetime, timedelta

# Custom function to handle datetime serialization
def custom_serializer(obj):
    if isinstance(obj, datetime):
        return obj.isoformat()  # Convert datetime to an ISO 8601 string
    raise TypeError(f"Type {type(obj)} not serializable")

def create_unified_transcription_object( start_time_str, recordings_dir="recordings"):
    """
    Creates a unified JSON-like object from transcription files in the recordings directory.
    Each segment includes the username and is sorted by absolute start time.

    Args:
        recordings_dir (str): Path to the 'recordings' directory.

    Returns:
        list: Sorted list of segment dictionaries with username, start, end, and text.
    """
    all_segments = []

    # Iterate over each user directory in 'recordings'
    for user_dir in os.listdir(recordings_dir):
        user_path = os.path.join(recordings_dir, user_dir)
        if not os.path.isdir(user_path):
            continue

        # Load metadata.json
        metadata_path = os.path.join(user_path, "metadata.json")
        if not os.path.exists(metadata_path):
            print(f"Warning: metadata.json not found in {user_path}")
            continue

        with open(metadata_path, 'r', encoding='utf-8') as f:
            metadata = json.load(f)

        # Process each recording in metadata
        for recording in metadata.get("recordings", []):
            username = recording.get("username")
            if not username:
                print(f"Warning: No username found in recording {recording}")
                continue

            if not start_time_str:
                print(f"Warning: No user_recording_start_time in {recording}")
                continue
            try:
                recording_start = datetime.strptime(start_time_str, "%Y-%m-%d %H:%M:%S")
            except ValueError as e:
                print(f"Error parsing time {start_time_str}: {e}")
                continue

            # Derive the adjusted transcription file name
            transcription_file = recording.get("transcription_file")
            if not transcription_file:
                print(f"Warning: No transcription_file in {recording}")
                continue
            # Replace '_transcription.json' with '_transcription_adjusted.json'
            adjusted_file = transcription_file.replace("_transcription.json", "_transcription_adjusted.json")
            adjusted_path = os.path.join(user_path, adjusted_file)

            # Load the transcription file
            if not os.path.exists(adjusted_path):
                print(f"Warning: {adjusted_file} not found in {user_path}")
                continue

            with open(adjusted_path, 'r', encoding='utf-8') as f:
                transcription = json.load(f)

            # Process each segment
            for segment in transcription.get("segments", []):
                rel_start = segment.get("start")
                rel_end = segment.get("end")
                text = segment.get("text")

                if rel_start is None or rel_end is None or not text:
                    print(f"Warning: Incomplete segment in {adjusted_file}: {segment}")
                    continue

                # Calculate absolute times
                abs_start = recording_start + timedelta(seconds=rel_start)
                abs_end = recording_start + timedelta(seconds=rel_end)

                # Create segment dictionary
                segment_dict = {
                    "username": username,
                    "start": abs_start,  # datetime object
                    "end": abs_end,      # datetime object
                    "text": text
                }
                all_segments.append(segment_dict)

    # Sort all segments by absolute start time
    all_segments.sort(key=lambda x: x["start"])

    return all_segments


unified_object = create_unified_transcription_object(start_time_str)
unified_obj_path =      base_dir + '/unified_object.json'
final_transcript_path = base_dir + '/transcript.txt'
# Save JSON with datetime converted to string
with open(unified_obj_path, 'w') as f:
    json.dump(unified_object, f, indent=2, default=custom_serializer)

##<font color='00A115'> Result

In [ ]:
#@title <font color='00A115'> json_to_text_transcript ()
from datetime import datetime

def json_to_text_transcript(data, WebVTT=False):
    # List to store groups of consecutive entries
    groups = []
    current_group = None

    # Step 1: Group consecutive entries by speaker
    for entry in data:
        if current_group is None or entry['username'] == current_group['speaker']:
            # If no current group exists, start a new one
            if current_group is None:
                current_group = {
                    'speaker': entry['username'],
                    'start_time': entry['start'],
                    'end_time': entry['end'],
                    'texts': [entry['text'].strip()]
                }
            # If speaker is the same, append text to current group
            else:
                current_group['texts'].append(entry['text'].strip())
        else:
            # Speaker changed, save current group and start a new one
            groups.append(current_group)
            current_group = {
                'speaker': entry['username'],
                'start_time': entry['start'],
                'end_time': entry['end'],
                'texts': [entry['text'].strip()]
            }

    # Don’t forget to append the last group
    if current_group is not None:
        groups.append(current_group)

    # Step 2: Format each group into transcript lines
    transcript_lines = []
    for i, group in enumerate(groups, 1):  # Start numbering at 1
        # Convert the group's start time to HH:MM:SS format
        start_time = datetime.fromisoformat(group['start_time']).strftime("%H:%M:%S")
        end_time = datetime.fromisoformat(group['end_time']).strftime("%H:%M:%S")

        # Format the entry: number, [user @ time], and all texts joined with spaces
        # entry_text = f"{i} [{group['speaker']} @ {start_time}]\n{' '.join(group['texts'])}\n\n"
        if WebVTT:
            entry_text = f"{start_time} --> {end_time}\n{group['speaker']}: {' '.join(group['texts'])}\n"
        else:
            entry_text = f"{i} {group['speaker']} {start_time}\n{' '.join(group['texts'])}\n\n"

        transcript_lines.append(entry_text)

    # Step 3: Join all lines with newlines
    return "\n".join(transcript_lines)

# ==============================================================================
# Example usage with the rest of your code
# Load JSON back into a Python object
with open(unified_obj_path, 'r') as f:
    unified_object = json.load(f)

# Generate transcript
transcript = json_to_text_transcript(unified_object, WebVTT=False)

# Save transcript to file
with open(final_transcript_path, 'w') as f:
    f.write(transcript)

# Print for verification
print(f"{CYAN}\nTransformed transcript saved as 'transcript.txt':{RESET}")
print(transcript)


Transformed transcript saved as 'transcript.txt':
1 runfishrun 17:46:23
Yeah, I don't know what's your stream on here.


2 Unknown 17:46:27
No, it's okay. Just working on the video, you know, trying to dissolve and put some sliders in a better position.


3 runfishrun 17:46:39
Yeah, so how is that going?


4 Unknown 17:46:42
I'm currently helping myself with ChatGPT due to it's new for me. But I'm seriously thinking to change a different program after this. Like that's the one and only project I'm doing with Canvas. It's like because when you have slides and then you have you make different slides in between those slides, there's a millisecond of a screen flicker which quickly flashes and like it's really making a weird behavior. So it's asking, it's telling me to dissolve in between with a little plus, but there is no plus. Like GPT tells me clearly there's a little plus, but that's what it is. Like, yeah. What are you working on?


5 runfishrun 17:47:29
Yeah, I just get this bot don

In [ ]:
#@title 💾 zip recordins_and_transcripts

'''
For downloads we zip it.
(Google Colab doesn't support download and upload of directories.)
'''
from datetime import datetime
# Get current date in yymmdd format
current_date = datetime.now().strftime("%y%m%d")

# Create base zip filename with date prefix
base_zip_filename = f"{current_date}_recordings.zip"

# Check if file exists and add version number if needed
i = 1
zip_filename = base_zip_filename
while os.path.exists(zip_filename):
    zip_filename = f"{current_date}_recordings_and_transcripts{i}.zip"
    i += 1

# Zip the recordings directory with the dated filename
!zip -r {zip_filename} recordings

print(f"{''}Created zip file: {zip_filename}{''}")

  adding: recordings/ (stored 0%)
  adding: recordings/runfishrun/ (stored 0%)
  adding: recordings/runfishrun/session_20250514_17M51_transcription.json (deflated 91%)
  adding: recordings/runfishrun/session_20250514_17M51_cleaned.wav (deflated 70%)
  adding: recordings/runfishrun/metadata.json (deflated 59%)
  adding: recordings/runfishrun/session_20250514_17M51.wav (deflated 80%)
  adding: recordings/runfishrun/session_20250514_17M51_transcription_adjusted.json (deflated 91%)
  adding: recordings/unified_object.json (deflated 78%)
  adding: recordings/transcript.txt (deflated 59%)
  adding: recordings/Unknown/ (stored 0%)
  adding: recordings/Unknown/session_20250514_17M51_transcription.json (deflated 91%)
  adding: recordings/Unknown/session_20250514_17M51_cleaned.wav (deflated 56%)
  adding: recordings/Unknown/metadata.json (deflated 59%)
  adding: recordings/Unknown/session_20250514_17M51.wav (deflated 72%)
  adding: recordings/Unknown/session_20250514_17M51_transcription_adjusted

In [ ]:
# @title Bebugging: json_to_text_transcript ()

import json
from datetime import datetime

# Custom function to handle datetime serialization
def custom_serializer(obj):
    if isinstance(obj, datetime):
        return obj.isoformat()  # Convert datetime to an ISO 8601 string
    raise TypeError(f"Type {type(obj)} not serializable")

# Save JSON with datetime converted to string
with open('unified_object.json', 'w') as f:
    json.dump(unified_object, f, indent=2, default=custom_serializer)


import json

# Load JSON back into a Python object
with open('unified_object.json', 'r') as f:
    unified_object = json.load(f)


# Transform to simplified transcript format
def json_to_text_transcript(data):
    transcript_lines = []
    for i, entry in enumerate(data, 1):
        # Convert start time to hh:mm:ss format
        start_time = datetime.fromisoformat(entry["start"]).strftime("%H:%M:%S")
        # Create entry with username and start time in header
        entry_text = f"{i}\n"
        entry_text += f"[{entry['username']} @ {start_time}]\n"
        entry_text += f"{entry['text'].strip()}\n"
        transcript_lines.append(entry_text)
    return "\n".join(transcript_lines)

# Generate transcript
transcript = json_to_text_transcript(unified_object)

# # Save transcript to file
# with open('transcript.txt', 'w') as f:
#     f.write(transcript)

print(transcript)

1
[runfishrun @ 17:46:23]
Yeah, I don't know what's your stream on here.

2
[Unknown @ 17:46:27]
No, it's okay.

3
[Unknown @ 17:46:28]
Just working on the video, you know, trying to dissolve and put some sliders in a better position.

4
[runfishrun @ 17:46:39]
Yeah, so how is that going?

5
[Unknown @ 17:46:42]
I'm currently helping myself with ChatGPT due to it's new for me.

6
[Unknown @ 17:46:48]
But I'm seriously thinking to change a different program after this.

7
[Unknown @ 17:46:52]
Like that's the one and only project I'm doing with Canvas.

8
[Unknown @ 17:46:57]
It's like because when you have slides and then you have you make different slides in between those slides, there's a millisecond of a screen flicker which quickly flashes and like it's really making a weird behavior.

9
[Unknown @ 17:47:14]
So it's asking, it's telling me to dissolve in between with a little plus, but there is no plus.

10
[Unknown @ 17:47:21]
Like GPT tells me clearly there's a little plus, but th